In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Step 1: Read the Excel file
file_path = r"file_path"
data = pd.read_excel(file_path)

# Ensure the column names in the Excel file match the database table structure
# For example, check if columns are ['last_name', 'first_name', 'collected_or_harvested', 'harvesting_process', 'harvest_responsibility', 'labourers_count']

# Step 2: Establish connection to the MySQL database
try:
    connection = mysql.connector.connect(
        host='localhost',  # Change to your host
        user='root',  # Replace with your MySQL username
        password='*****',  # Replace with your MySQL password
        database='*****'  # Replace with your database name
    )

    if connection.is_connected():
        cursor = connection.cursor(dictionary=True)

        # Step 3: Retrieve farmer IDs based on last_name and first_name
        for index, row in data.iterrows():
            last_name = row['last_name']
            first_name = row['first_name']

            # Query to get farmer_id
            cursor.execute(
                """
                SELECT farmer_id FROM farmer WHERE last_name = %s AND first_name = %s
                """, (last_name, first_name)
            )
            result = cursor.fetchone()

            if result:
                farmer_id = result['farmer_id']
                
                # Step 4: Prepare the insert statement
                insert_query = """
                INSERT INTO harvesting (
                    farmer_id, collected_or_harvested, harvesting_process, harvest_responsibility, labourers_count
                ) VALUES (%s, %s, %s, %s, %s)
                """

                cursor.execute(
                    insert_query,
                    (
                        farmer_id,
                        row['collected_or_harvested'] if pd.notna(row['collected_or_harvested']) else None,
                        row['harvesting_process'] if pd.notna(row['harvesting_process']) else None,
                        row['harvest_responsibility'] if pd.notna(row['harvest_responsibility']) else None,
                        row['labourers_count'] if pd.notna(row['labourers_count']) else None
                    )
                )

        # Step 5: Commit the transaction
        connection.commit()
        print("Data imported successfully!")

except Error as e:
    print(f"Error: {e}")

finally:
    # Step 6: Close the connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")
